# Skill Check 10

The block below imports the necessary packages.

In [1]:
import numpy as np
import pandas as pd

In this assignment, you will work with the Dow dataset you have handled a few weeks ago. Reading and cleaning the dataset is done in the block below.

In [2]:
df = pd.read_excel('impurity_dataset-training.xlsx')

def is_real_and_finite(x):
    if not np.isreal(x):
        return False
    elif not np.isfinite(x):
        return False
    else:
        return True
    
all_data = df[df.columns[1:]].values
numeric_map = df[df.columns[1:]].applymap(is_real_and_finite)
real_rows = numeric_map.all(axis = 1).copy().values
X_dow = np.array(all_data[real_rows, :-5], dtype = 'float')
y_dow = np.array(all_data[real_rows, -3], dtype = 'float')
y = y_dow.reshape(-1, 1)

## 1. k-Means (35 pts)

The k-means algorithm is an iterative type of algorithm that pursues the following expectation-maximization:

- **Expectation**: expect that the points close to the center of a cluster belong to that cluster
- **Maximization**: maximize the proximity of points to the center of a cluster by moving the center

In this problem, you will implement the k-means algorithm with several functions provided below.

In [3]:
def dist(pt1, pt2):
    return np.sqrt(sum([(xi - yi)**2 for xi, yi in zip(pt1, pt2)]))

def expected_assignment(pt, cluster_centers):
    dists = [dist(pt, ci) for ci in cluster_centers]
    min_index = dists.index(min(dists))
    return min_index

def new_centers(cluster_points, centers):
    centers = list(centers)
    for i, ci in enumerate(cluster_points):
        if ci != []:
            centers[i] = np.mean(ci, axis = 0)
    return centers

### 1a: Data Preparation (15 pts)

First, you need a preprocessing step to transform the given Dow dataset. Obtain a matrix named `X` as a result of the following steps:

- Standardize `X_dow` using the "standard scaler"
- Project the standardized `X_dow` onto 2-dimensional space using PCA
- Take every 5th data point

The resulting matrix should be stored as `X`.

In [4]:
########################################
# Start your code here
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

X_scaled = StandardScaler().fit_transform(X_dow)
X_pca = PCA(n_components = 2).fit_transform(X_scaled)
X = X_pca[::5]
########################################

In [5]:
assert X.shape[1] == 2, "projection onto the wrong dimension"
assert X.shape[0] == 2060, "wrong data size"
assert np.isclose(X[:, 0].sum(), 20.5537312894985)

### 1b: One iteration of k-means (10 pts)

Making use of the functions defined above, apply the k-means algorithm **once** to the dataset and report the resulting new cluster centers in the variable `new_cluster_centers`. The initial guesses are provided below as `old_cluster_centers`, and you should keep the same data structure (e.g. a list of lists) as `new_cluster_centers`.

In [6]:
old_cluster_centers = ([0, 0], [2, 2], [30, -2])

In [7]:
########################################
# Start your code here
old_centers = np.array(old_cluster_centers)

clusters = []
for i in range(old_centers.shape[0]):
    clusters.append([])
    
for pt in X:
    cluster_idx = expected_assignment(pt, old_centers)
    clusters[cluster_idx].append(pt)
    
new_cluster_centers = new_centers(clusters, old_centers)
########################################

In [8]:
assert np.isclose(np.array(new_cluster_centers).sum(axis = 0)[0], 29.21008259)
assert np.isclose(np.array(new_cluster_centers).mean(axis = 1)[1], 2.22463995)

### 1c: Convergence of k-means (10 pts)

In this problem, you will define a function `kmeans` that implements the k-means alogrithm until the pre-defined convergence criterion is satistified. The `kmeans` function takes the following arguments:

- `X`: dataset (numpy array)
- `centers`: initial guesses for cluster centers (numpy array)
- `tol`: convergence tolerance (float, default = 0.1)

Note that the number of clusters will be implicitly defined by the number of rows in the `centers` array.

**Convergence criterion**: the maximum change in distance between cluster centers < `tol`.

Your `kmeans` function should return `new_center` (numpy array) and `clusters` (list of lists with the indices of the points in each cluster).

In [9]:
def kmeans(X, centers, tol = 0.1):
########################################
# Start your code here
    old_center = np.array(centers)
    max_dist = 1e6
    
    while max_dist > tol:
        clusters = []
        for i in range(old_center.shape[0]):
            clusters.append([])
            
        for pt in X:
            cluster_idx = expected_assignment(pt, old_center)
            clusters[cluster_idx].append(pt)
            
        new_center = new_centers(clusters, old_center)
        
        max_dist = 0
        for i in range(old_center.shape[0]):
            dist = np.linalg.norm(old_center[i] - new_center[i])
            if max_dist < dist:
                max_dist = dist
                
        old_center = np.array(new_center)
########################################
    return np.array(new_center), clusters

In [10]:
centers, clusters = kmeans(X, np.array(old_cluster_centers))

assert type(centers) == np.ndarray, "wrong data type for centers"
assert type(clusters) == list, "wrong data type for clusters"
assert np.isclose(centers.mean(axis = 1)[2], 13.04660452), "wrong cluster centers"
assert len(clusters[0]) == 1403, "wrong clusters"

## 2. Mean Shift (35 pts)

The mean shift algorithm is also an iterative type of algorithm, but unlike the k-means, it is density-based rather than expectation-maximization based. You may find the functions defined below useful through this problem.

In [11]:
def get_distance(x1, x2):
    return np.linalg.norm(x1 - x2, 2)

def get_nearby_points(x, x_list, bandwidth):
    dist_pairs = []
    for i, xi in enumerate(x_list):
        dist = get_distance(x, xi)
        dist_pairs.append([dist, i, xi])
    in_window = [pt[-1] for pt in dist_pairs if pt[0] <= bandwidth]
    return in_window

def get_new_centroid(old_centroid, x_list, bandwidth):
    in_range = get_nearby_points(old_centroid, x_list, bandwidth)
    if len(in_range) == 0:
        new_centroid = old_centroid
    else:
        new_centroid = np.array(in_range).mean(axis = 0)
    return new_centroid

### 2a: One iteration of mean shift (15 pts)

Take `old_cluster_centers` (defined in Problem 1) as intial guesses for centroids and find a new centroids `new_centroids` using a `bandwith` of 5.

In [12]:
########################################
# Start your code here
new_centroids = []
for c in old_cluster_centers:
    new_centroids.append(get_new_centroid(c, X, 5))
########################################

In [13]:
assert np.isclose(np.array(new_centroids).sum(axis = 0)[0], 29.43546375)
assert np.isclose(np.linalg.norm(new_centroids), 30.992548441545583)

### 2b: Converging mean shift (20 pts)

A typical convergence criterion for the mean shift algorithm is checking whether old centroids and new centroids are identical to each other. Report the final centroids `final_centroids` after convergence (15 pts) and the number of iterations needed to reach convergence `num_iter` (5 pts).

In [14]:
########################################
# Start your code here
delta = 1
old_centroids = old_cluster_centers

while delta > 0:
    new_centroids = []
    for c in old_centroids:
        new_centroids.append(get_new_centroid(c, X, 5))
        
    delta = 0
    for i, c in enumerate(old_centroids):
        if delta < get_distance(c, new_centroids[i]):
            delta = 1
            
    old_centroids = new_centroids
    
final_centroids = new_centroids
########################################

In [15]:
assert np.isclose(np.array(final_centroids).sum(), 24.840886688211803)
assert np.isclose((final_centroids[1] - final_centroids[2])[0], -31.595831127643677)

In [16]:
assert np.array(old_cluster_centers * 4).sum(axis = 0)[0] == 128

## 3. Gaussian Mixture Models (30 pts)

The Gaussian Mixture Models (GMM) is a generative algorithm based on expectation-maximization. In this problem, you will play with the built-in `sklearn.mixture.GaussainMixture` object.

### 3a: Import the object (10 pts)

Import the `sklearn.mixture.GaussianMixture` object.

In [17]:
########################################
# Start your code here
from sklearn.mixture import GaussianMixture
########################################

In [18]:
assert GaussianMixture.__init__

### 3b: Silhouette score and Calinski-Harabasz score (10 pts)

Apply the GMM to `X` with the following settings and report the resulting silhouette score `sil_score` and Calinski-Harabasz score `ch_score`.

- `n_components`: 4
- `random_state`: 42
- `covariance_type`: `tied`

In [19]:
########################################
# Start your code here
gmm = GaussianMixture(n_components = 4, random_state = 42, covariance_type = 'tied')
gmm.fit(X)
y_predict = gmm.predict(X)

from sklearn.metrics import silhouette_score, calinski_harabasz_score

sil_score = silhouette_score(X, y_predict)
ch_score = calinski_harabasz_score(X, y_predict)
########################################

In [20]:
assert np.isclose(sil_score * ch_score, 5086.217116738762)

### 3c: Mixed membership (10 pts)

GMM supports mixed membership, so points have probabilites of belonging to all clusters. Find the cluster that has the second highest of probability for 101st data point in `X` (i.e. the point indexed by 100). Save the cluster label to `sec_high_cluster` and the corresponding probabiltiy `sec_high_prob`. You should assume that clusters are labeled as 1-4 (i.e. there is no cluster 0).

In [21]:
########################################
# Start your code here
prob = gmm.predict_proba(X)[100, :]
prob_copy = prob.copy()

prob.sort()
sec_high_prob = prob[-2]
sec_high_cluster = np.where(prob_copy == sec_high_prob)[0] + 1
########################################

In [22]:
assert np.isclose(sec_high_prob * sec_high_cluster, 0.005392117243474652)